In [1]:
from tqdm import tqdm
import torch.nn.functional as F
import torch
import torch.optim as optim
from torchsummary import summary
import torch.nn as nn
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.optim.lr_scheduler import OneCycleLR
import model_1, model_2, model_3

In [2]:
SEED = 1
cuda = torch.cuda.is_available()
torch.manual_seed(SEED)
if cuda:
    torch.cuda.manual_seed(SEED)

In [3]:
def data_loader_preprocessing():
# Train Phase transformations
    train_transforms = transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,))
                                       ])

    # Test Phase transformations
    test_transforms = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.1307,), (0.3081,))
                                        ])


    train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
    test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)



    # dataloader arguments - something you'll fetch these from cmdprmt
    dataloader_args = dict(shuffle=True, batch_size=32, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

    # train dataloader
    train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

    # test dataloader
    test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

    return train_loader, test_loader

In [4]:
train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):

    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm

    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

    test_acc.append(100. * correct / len(test_loader.dataset))


train_loader, test_loader = data_loader_preprocessing()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

100%|██████████| 9.91M/9.91M [00:00<00:00, 19.0MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 492kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 4.65MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 10.5MB/s]
/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:627: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


### Model 1 Training and Logs

In [ ]:
model =  model_1.Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
print(summary(model, input_size=(1, 28, 28)))
print('Training the model...')
print('-'*20)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
            Conv2d-3           [-1, 12, 28, 28]             864
              ReLU-4           [-1, 12, 28, 28]               0
         MaxPool2d-5           [-1, 12, 14, 14]               0
            Conv2d-6           [-1, 16, 14, 14]           1,728
              ReLU-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 16, 12, 12]           2,304
              ReLU-9           [-1, 16, 12, 12]               0
        MaxPool2d-10             [-1, 16, 6, 6]               0
           Conv2d-11             [-1, 16, 4, 4]           2,304
             ReLU-12             [-1, 16, 4, 4]               0
           Conv2d-13             [-1, 16, 2, 2]           2,304
             ReLU-14             [-1, 1

Loss=0.14111807942390442 Batch_id=1874 Accuracy=82.53: 100%|██████████| 1875/1875 [00:21<00:00, 86.67it/s]



Test set: Average loss: 0.1046, Accuracy: 9688/10000 (96.88%)

EPOCH: 1


Loss=0.0983821228146553 Batch_id=1874 Accuracy=97.71: 100%|██████████| 1875/1875 [00:20<00:00, 91.15it/s]



Test set: Average loss: 0.0568, Accuracy: 9818/10000 (98.18%)

EPOCH: 2


...


Test set: Average loss: 0.0423, Accuracy: 9874/10000 (98.74%)

EPOCH: 13


Loss=1.967899879673496e-05 Batch_id=1874 Accuracy=99.36: 100%|██████████| 1875/1875 [00:20<00:00, 90.11it/s]



Test set: Average loss: 0.0344, Accuracy: 9912/10000 (99.12%)

EPOCH: 14


Loss=2.8345180908218026e-05 Batch_id=1874 Accuracy=99.37: 100%|██████████| 1875/1875 [00:21<00:00, 88.75it/s]



Test set: Average loss: 0.0332, Accuracy: 9914/10000 (99.14%)



## Model - 2 Training and Logs

In [ ]:
model =  model_2.Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
print(summary(model, input_size=(1, 28, 28)))
print('Training the model...')
print('-'*20)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5           [-1, 12, 28, 28]             864
              ReLU-6           [-1, 12, 28, 28]               0
       BatchNorm2d-7           [-1, 12, 28, 28]              24
           Dropout-8           [-1, 12, 28, 28]               0
         MaxPool2d-9           [-1, 12, 14, 14]               0
           Conv2d-10           [-1, 16, 14, 14]           1,728
             ReLU-11           [-1, 16, 14, 14]               0
      BatchNorm2d-12           [-1, 16, 14, 14]              32
          Dropout-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

Loss=0.14013588428497314 Batch_id=1874 Accuracy=95.36: 100%|██████████| 1875/1875 [00:24<00:00, 77.27it/s]



Test set: Average loss: 0.0423, Accuracy: 9869/10000 (98.69%)

EPOCH: 1


Loss=0.006970836315304041 Batch_id=1874 Accuracy=98.20: 100%|██████████| 1875/1875 [00:23<00:00, 79.09it/s]



Test set: Average loss: 0.0321, Accuracy: 9895/10000 (98.95%)

EPOCH: 2


...


Test set: Average loss: 0.0223, Accuracy: 9927/10000 (99.27%)

EPOCH: 13


Loss=0.08499334007501602 Batch_id=1874 Accuracy=99.04: 100%|██████████| 1875/1875 [00:25<00:00, 72.75it/s]



Test set: Average loss: 0.0226, Accuracy: 9925/10000 (99.25%)

EPOCH: 14


Loss=0.0008189385407604277 Batch_id=1874 Accuracy=99.11: 100%|██████████| 1875/1875 [00:25<00:00, 74.37it/s]



Test set: Average loss: 0.0199, Accuracy: 9940/10000 (99.40%)



## Model - 3 Training and Logs

In [5]:
def data_loader_preprocessing():
# Train Phase transformations
    train_transforms = transforms.Compose([
                                      #  transforms.Resize((28, 28)),
                                      #  transforms.ColorJitter(brightness=0.10, contrast=0.1, saturation=0.10, hue=0.1),
                                       transforms.RandomRotation((-10.0, 10.0), fill=(1,)),
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,), (0.3081,)) # The mean and std have to be sequences (e.g., tuples), therefore you should add a comma after the values.
                                       # Note the difference between (0.1307) and (0.1307,)
                                       ])

    # Test Phase transformations
    test_transforms = transforms.Compose([
                                        transforms.ToTensor(),
                                        transforms.Normalize((0.1307,), (0.3081,))
                                        ])


    train = datasets.MNIST('./data', train=True, download=True, transform=train_transforms)
    test = datasets.MNIST('./data', train=False, download=True, transform=test_transforms)



    # dataloader arguments - something you'll fetch these from cmdprmt
    dataloader_args = dict(shuffle=True, batch_size=32, num_workers=4, pin_memory=True) if cuda else dict(shuffle=True, batch_size=64)

    # train dataloader
    train_loader = torch.utils.data.DataLoader(train, **dataloader_args)

    # test dataloader
    test_loader = torch.utils.data.DataLoader(test, **dataloader_args)

    return train_loader, test_loader


train_loader, test_loader = data_loader_preprocessing()
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

In [6]:
model =  model_3.Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = StepLR(optimizer, step_size=8, gamma=0.1)
print(summary(model, input_size=(1, 28, 28)))
print('Training the model...')
print('-'*20)
EPOCHS = 15
for epoch in range(EPOCHS):
    print("EPOCH:", epoch)
    train(model, device, train_loader, optimizer, epoch)
    scheduler.step()
    test(model, device, test_loader)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              72
              ReLU-2            [-1, 8, 28, 28]               0
       BatchNorm2d-3            [-1, 8, 28, 28]              16
           Dropout-4            [-1, 8, 28, 28]               0
            Conv2d-5           [-1, 12, 28, 28]             864
              ReLU-6           [-1, 12, 28, 28]               0
       BatchNorm2d-7           [-1, 12, 28, 28]              24
           Dropout-8           [-1, 12, 28, 28]               0
         MaxPool2d-9           [-1, 12, 14, 14]               0
           Conv2d-10           [-1, 16, 14, 14]           1,728
             ReLU-11           [-1, 16, 14, 14]               0
      BatchNorm2d-12           [-1, 16, 14, 14]              32
          Dropout-13           [-1, 16, 14, 14]               0
           Conv2d-14           [-1, 16,

Loss=0.05117357522249222 Batch_id=1874 Accuracy=93.83: 100%|██████████| 1875/1875 [00:28<00:00, 66.49it/s]



Test set: Average loss: 0.0461, Accuracy: 9867/10000 (98.67%)

EPOCH: 1


Loss=0.023698857054114342 Batch_id=1874 Accuracy=97.29: 100%|██████████| 1875/1875 [00:27<00:00, 67.80it/s]



Test set: Average loss: 0.0338, Accuracy: 9890/10000 (98.90%)

EPOCH: 2


...


Test set: Average loss: 0.0193, Accuracy: 9942/10000 (99.42%)

EPOCH: 13


Loss=0.005125211086124182 Batch_id=1874 Accuracy=98.71: 100%|██████████| 1875/1875 [00:27<00:00, 67.64it/s]



Test set: Average loss: 0.0187, Accuracy: 9945/10000 (99.45%)

EPOCH: 14


Loss=0.2332158088684082 Batch_id=1874 Accuracy=98.68: 100%|██████████| 1875/1875 [00:27<00:00, 68.38it/s]



Test set: Average loss: 0.0182, Accuracy: 9942/10000 (99.42%)

